### Edit các thông tin ở đây

In [ ]:
# @title ID { display-mode: "form" }
start_id = 13807 # @param {type:"number"}
end_id   = 14500 # @param {type:"number"}

1. Ấn cái icon "chìa khóa" bên trái và thêm/+Add new secret `USER_CODE`, `TOKEN_CODE` (lấy từ UserCode, TokenCode của 1 url ảnh trên ctsv)
2. Chỉnh id hoạt động bắt đầu, kết thúc: `start_id`, `end_id` ở trên
3. Runtime -> Run all (hoặc Ctrl + F9)
4. Đợi cho chạy xong (5p)
5. Ấn cái icon "folder" bên trái và tải file DanhSachHoatDong.xlsx về

Note: File errors.csv chứa id của hoạt động không lấy đc

### Code chạy

#### Definition

In [ ]:
%pip install html2text

In [ ]:
from google.colab import userdata
CTSVTokenCode = userdata.get('TOKEN_CODE')
CTSVUserNameCode = userdata.get('USER_CODE')

import asyncio
import aiohttp
from aiohttp import ClientSession

connector = aiohttp.TCPConnector(limit_per_host=50)
custom_client = aiohttp.ClientSession(connector=connector)

In [ ]:
import json
import html2text

# HTML to Text conversion
html_converter = html2text.HTML2Text()
html_converter.ignore_images = True
html_converter.unicode_snob = True
html_converter.ignore_links = False

# Error list
errored_list = []

# Process API Response
def process_data(activity_id: int, data):
    if data["RespCode"] == 0:
        activity = data["Activities"][0]
        print(f"GET {activity_id}: {activity['StartTime']} {activity['FinishTime']} {activity['AName']}")
        activity["ADesc"] = html_converter.handle(activity["ADesc"])
        return activity
    else:
        print(f"GET {activity_id} failed: {data['RespCode']} {data['RespText']}")

        # 205 Hoạt động không tồn tại
        if data["RespCode"] != 205:
            errored_list.append(activity_id)

        return None

# Make API Request
async def make_request(session: ClientSession, activity_id: int):
    url = "https://ctsv.hust.edu.vn/api-t/Activity/GetActivityById"
    request_body = {
        "AId": activity_id,
        "TokenCode": CTSVTokenCode,
        "UserName": CTSVUserNameCode
    }

    try:
        async with session.post(url, json=request_body) as response:
            if response.status != 200:
                raise Exception(f"HTTP error! Status: {response.status}")
            data = await response.json()

            return process_data(activity_id, data)
    except Exception as e:
        print(f"Error making request for {activity_id}: {str(e)}")
        return None

# Send Multiple Requests Concurrently
async def send_requests(start_id: int, end_id: int):
    results = []

    async with custom_client as session:
        tasks = [make_request(session, activity_id) for activity_id in range(start_id, end_id + 1)]
        responses = await asyncio.gather(*tasks)

        for response in responses:
            if response is not None:
                results.append(response)

        # Save Results to Files
        with open('results.json', 'w', encoding='utf-8') as f:
            json.dump([activity for activity in results], f, ensure_ascii=False, indent=2)
        print(f"Successfully processed {len(results)} results")

        with open('errors.csv', 'w', encoding='utf-8') as f:
            f.write("\n".join(map(str, errored_list)))
        print(f"Errors written to errors.csv")

    return results

#### Run code

In [ ]:
async with aiohttp.ClientSession() as session:
    test = await make_request(session, 12584)
    if (test == None):
        quit()
        raise Exception("Lỗi Token")

activities = await send_requests(start_id, end_id)


GET 12584: 2024-12-31 00:00:00 2025-01-07 00:00:00 Thành viên Tổ Hỗ trợ triển khai Hệ thống iCTSV 2024.1
GET 13834: 2025-03-12 18:00:00 2025-03-12 21:00:00 2024.2 Thành viên BTC và TNV hỗ trợ chương trình “Giao thoa - SEEE”
GET 13819: 2024-12-28 00:00:00 2025-03-15 00:00:00 Tham gia Cuộc thi Ý tưởng Sáng tạo Sinh viên 2025
GET 13835: 2025-03-03 00:00:00 2025-06-30 00:00:00 Xây dựng kế hoạch học tập và chi tiêu cho học kỳ 2024.2
GET 13833: 2025-03-12 18:00:00 2025-03-12 21:00:00 2024.2 Tham gia trình diễn tại concert “Xuân Hạ Thu Đông Rồi Lại Xuân”
GET 13815: 2025-03-08 00:00:00 2025-03-09 23:59:00 Tham gia Giải Cờ tướng – Cờ úp mở rộng HXC Cup IV
GET 13851 failed: 205 Hoạt động không tồn tại
GET 13850 failed: 205 Hoạt động không tồn tại
GET 13811: 2025-02-26 08:00:00 2025-03-07 18:00:00 Tham gia Tuần sách SME kỳ 2024.2
GET 13825: 2025-03-18 18:00:00 2025-03-18 20:30:00 BKD GEN 12 - SINH HOẠT HÀNG TUẦN - 18/3/2025
GET 13852 failed: 205 Hoạt động không tồn tại
GET 13848 failed: 205 Hoạt 

In [ ]:
if(len(activities) == 0):
    raise Exception("EMPTY ACTIVITY LIST. CHECK start_id and end_id.")

### Data Analysis below

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(activities)

In [ ]:
columns_to_remove = ['ACode','GId','AGId','Data','AStatus','UAStatus','ARefId','ACriteriaLst','UserRole','Publish']
df_fixed = df.drop(columns=columns_to_remove)
df_fixed['CriteriaLst'] = df_fixed['CriteriaLst'].apply(lambda criterias: '\n'.join([item['CName'] for item in criterias]))
df_fixed['Link'] = df_fixed['AId'].apply(lambda x: f"https://ctsv.hust.edu.vn/#/hoat-dong/{x}/chi-tiet")

# Reordering
column_order = [
    'AId', 'AName', 'GName','CriteriaLst',
    'CreateDate', 'Deadline', 'Link', 'CreateUser',
    'StartTime', 'FinishTime', 'ADesc', 'APlace',
    'AType', 'AGDesc',
    'Avatar',
]
df_fixed = df_fixed[column_order]

# Rename columns
df_fixed = df_fixed.rename(columns={
    'AName': 'Tên hoạt động',
    'AType': 'Loại HĐ',
    'ADesc': 'Nội dung',
    'StartTime': 'TG bắt đầu',
    'FinishTime': 'TG kết thúc',
    'APlace': 'Địa điểm',
    'GName': 'BTC',
    'Deadline': 'Hạn nộp MC',
    'CriteriaLst': 'DS tiêu chí',
    'CreateDate': 'TG tạo HĐ',
    'CreateUser': 'DS kiến nghị'
})



df_fixed


,AId,Tên hoạt động,BTC,DS tiêu chí,TG tạo HĐ,Hạn nộp MC,Link,DS kiến nghị,TG bắt đầu,TG kết thúc,Nội dung,Địa điểm,Loại HĐ,AGDesc,Avatar
0,13807,BTC và Kì thủ tham gia Giải Cờ tướng - Cờ úp H...,CLB Cờ tướng Bách khoa Hà Nội,"Thành viên CLB/Tổ/Đội/Nhóm SV hợp pháp, hoạt đ...",2025-02-20 11:25:26,2025-03-09 23:59:00,https://ctsv.hust.edu.vn/#/hoat-dong/13807/chi...,duy.pd235588@sis.hust.edu.vn,2025-03-08 07:00:00,2025-03-09 18:00:00,BTC và Kì thủ tham gia Giải Cờ tướng mở rộng H...,Đại học Bách khoa Hà Nội,Sinh hoạt câu lạc bộ,Sinh hoạt câu lạc bộ,
1,13808,Chương trình triển Lãm quốc tế Vietship lần th...,"Tổ hỗ trợ thực tập, việc làm doanh nghiệp","Tham gia hội thảo tăng cường kỹ năng mềm, phươ...",2025-02-25 16:05:09,2025-03-06 16:30:00,https://ctsv.hust.edu.vn/#/hoat-dong/13808/chi...,tien.nguyenviet@hust.edu.vn,2025-03-06 13:00:00,2025-03-06 16:30:00,Vietship 2025 là nơi hội tụ và trình diễn các ...,Trung tâm Hội nghị Quốc gia (NCC) (Mễ Trì – Na...,Sinh hoạt chuyên đề,Sinh hoạt chuyên đề,UploadFile/CTSV/DownloadImageAcitivity?AId=13808
2,13809,[SCLS] Cổ vũ vòng chung kết BKCUP 2025 đội bón...,Trường Hóa và Khoa học Sự sống,Tham gia cổ vũ các hoạt động phong trào văn hó...,2025-02-25 23:10:38,2025-02-26 10:45:00,https://ctsv.hust.edu.vn/#/hoat-dong/13809/chi...,thuy.ttm221376@sis.hust.edu.vn,2025-02-26 09:55:00,2025-02-26 10:45:00,"Sau những trận đấu nảy lửa, những đội bóng mạn...",SVD Bách khoa,Hoạt động thể thao,Hoạt động thể thao,
3,13810,Bồi dưỡng nội bộ,CLB Kế toán Tài chính - SEM,"Thành viên CLB học thuật, Lab nghiên cứu, Nhóm...",2025-02-26 08:41:45,2025-02-26 21:00:00,https://ctsv.hust.edu.vn/#/hoat-dong/13810/chi...,"hong.ltt232885@sis.hust.edu.vn, chi.cm223339@s...",2025-02-26 18:00:00,2025-02-26 21:00:00,Bồi dưỡng nội bộ\n\n,,Sinh hoạt câu lạc bộ,Sinh hoạt câu lạc bộ,
4,13811,Tham gia Tuần sách SME kỳ 2024.2,Trường Cơ khí,"Tham gia các hoạt động phổ biến, tìm hiểu bản ...",2025-02-26 08:53:56,2025-03-15 00:00:00,https://ctsv.hust.edu.vn/#/hoat-dong/13811/chi...,"quan.nguyenminh@hust.edu.vn, duy.nv227762@sis....",2025-02-26 08:00:00,2025-03-07 18:00:00,Để đánh dấu cho sự trở lại đầy mạnh mẽ của “Tủ...,"M811, 813 - C7","Ủng hộ, quyên góp","Ủng hộ, quyên góp",
5,13812,Giải chạy “10000 bước chân vì sức khoẻ mỗi ngà...,Ban CTSV,Tham gia các giải thể thao được tổ chức bởi cá...,2025-02-26 15:30:26,2025-06-23 20:00:00,https://ctsv.hust.edu.vn/#/hoat-dong/13812/chi...,tien.nguyenviet@hust.edu.vn,2025-02-28 20:00:00,2025-06-23 20:00:00,Giải chạy “10000 bước chân vì sức khoẻ mỗi ngà...,App iCTSV,Hoạt động thể thao,Hoạt động thể thao,UploadFile/CTSV/DownloadImageAcitivity?AId=13812
6,13813,T27: 6/3/25 HTONL-Gỡ rối và Giải tỏa áp lực Họ...,"Tổ hỗ trợ, tư vấn học tập và tâm lý","Tham gia hội thảo tăng cường kỹ năng mềm, phươ...",2025-02-26 15:33:31,2025-04-06 11:00:00,https://ctsv.hust.edu.vn/#/hoat-dong/13813/chi...,anh.dolan@hust.edu.vn,2025-04-06 09:00:00,2025-04-06 11:00:00,Bạn chưa được nhận đồ án tốt nghiệp do chưa có...,MSTEAM,Sinh hoạt chuyên đề,Sinh hoạt chuyên đề,
7,13814,BTC và CTV Giải Cờ tướng – Cờ úp mở rộng HXC C...,Hội Sinh Viên,"Tham gia hoạt động tình nguyện (mùa hè xanh, t...",2025-02-27 10:11:42,2025-03-09 23:59:00,https://ctsv.hust.edu.vn/#/hoat-dong/13814/chi...,thanh.nh214092@sis.hust.edu.vn,2025-03-08 00:00:00,2025-03-09 23:59:00,**⮚****Nội dung hoạt động :** \n\nBTC và CTV ...,,Tình nguyện tại chỗ,Tình nguyện tại chỗ,
8,13815,Tham gia Giải Cờ tướng – Cờ úp mở rộng HXC Cup IV,Hội Sinh Viên,"Tham gia tuyên truyền chủ trương của Đảng, chí...",2025-02-27 10:17:12,2025-03-09 23:59:00,https://ctsv.hust.edu.vn/#/hoat-dong/13815/chi...,thanh.nh214092@sis.hust.edu.vn,2025-03-08 00:00:00,2025-03-09 23:59:00,_**⮚**_ _**Nội dung hoạt động :**_ \n\n**o**...,Đại học Bách khoa Hà Nội.,Hoạt động thể thao,Hoạt động thể thao,
9,13816,Hội thảo tuyển dụng cán bộ nguồn và cán bộ kế ...,"Tổ hỗ trợ thực tập, việc làm doanh nghiệp","Tham gia hội thảo tăng cường kỹ năng mềm, phươ...",2025-02-27 16:14:53,2025-03-06 11:30:00,htt

In [ ]:
df_fixed.to_excel(f"DanhSachHoatDong{start_id}-{end_id}.xlsx", index=False)

In [ ]:
df_fixed.to_csv(f"DanhSachHoatDong{start_id}-{end_id}.csv", index=False)

In [ ]:
df_filtered = df_fixed[df_fixed['BTC'] != "Ban CTSV"]
print(df_filtered.shape)

# Extract AId column to an array
filtered_aid_array = df_filtered['AId'].to_list()

(32, 15)
